# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 11:27AM,248053,19,TR-RIH-10062022,"GCH Granules USA, Inc.",Released
1,Oct 6 2022 11:18AM,248051,19,MSP214663,"Methapharm, Inc.",Released
2,Oct 6 2022 11:18AM,248051,19,MSP214668,"Methapharm, Inc.",Released
3,Oct 6 2022 11:14AM,248049,19,60018075,"GUSA Granules USA, Inc.",Released
4,Oct 6 2022 11:12AM,248042,10,8613727,Eywa Pharma Inc.,Released
5,Oct 6 2022 11:12AM,248042,10,8613774,Eywa Pharma Inc.,Released
6,Oct 6 2022 11:12AM,248042,10,8613775,Eywa Pharma Inc.,Released
7,Oct 6 2022 11:12AM,248042,10,8613776,Eywa Pharma Inc.,Released
8,Oct 6 2022 11:12AM,248042,10,8613777,Eywa Pharma Inc.,Released
9,Oct 6 2022 11:12AM,248042,10,8613778,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,248047,Released,6
22,248048,Released,1
23,248049,Released,1
24,248051,Released,2
25,248053,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248047,NaN,NaN,6.0
248048,NaN,NaN,1.0
248049,NaN,NaN,1.0
248051,NaN,NaN,2.0
248053,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0.0,15.0,29.0
247943,0.0,1.0,0.0
247972,7.0,6.0,1.0
247985,0.0,0.0,3.0
247991,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0,15,29
247943,0,1,0
247972,7,6,1
247985,0,0,3
247991,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,0,15,29
1,247943,0,1,0
2,247972,7,6,1
3,247985,0,0,3
4,247991,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,,15,29
1,247943,,1,
2,247972,7,6,1
3,247985,,,3
4,247991,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc."
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc."
3,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc."
4,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.
25,Oct 6 2022 11:10AM,248047,10,Bio-PRF
31,Oct 6 2022 11:07AM,248048,19,ACG North America LLC
32,Oct 6 2022 11:01AM,248046,19,"GUSA Granules USA, Inc."
33,Oct 6 2022 10:55AM,248045,12,Hush Hush
34,Oct 6 2022 10:54AM,248044,19,"GUSA Granules USA, Inc."
35,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc.",,,1
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc.",,,2
2,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",,,1
3,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,,,21
4,Oct 6 2022 11:10AM,248047,10,Bio-PRF,,,6
5,Oct 6 2022 11:07AM,248048,19,ACG North America LLC,,,1
6,Oct 6 2022 11:01AM,248046,19,"GUSA Granules USA, Inc.",,,1
7,Oct 6 2022 10:55AM,248045,12,Hush Hush,,1,
8,Oct 6 2022 10:54AM,248044,19,"GUSA Granules USA, Inc.",,,1
9,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc.",,3,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc.",1,,
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc.",2,,
2,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",1,,
3,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,21,,
4,Oct 6 2022 11:10AM,248047,10,Bio-PRF,6,,
5,Oct 6 2022 11:07AM,248048,19,ACG North America LLC,1,,
6,Oct 6 2022 11:01AM,248046,19,"GUSA Granules USA, Inc.",1,,
7,Oct 6 2022 10:55AM,248045,12,Hush Hush,,1,
8,Oct 6 2022 10:54AM,248044,19,"GUSA Granules USA, Inc.",1,,
9,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc.",5,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc.",1,,
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc.",2,,
2,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",1,,
3,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,21,,
4,Oct 6 2022 11:10AM,248047,10,Bio-PRF,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc.",2.0,NaN,NaN
2,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,21.0,NaN,NaN
4,Oct 6 2022 11:10AM,248047,10,Bio-PRF,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 11:27AM,248053,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Oct 6 2022 11:18AM,248051,19,"Methapharm, Inc.",2.0,0.0,0.0
2,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,21.0,0.0,0.0
4,Oct 6 2022 11:10AM,248047,10,Bio-PRF,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1736086,65.0,11.0,8.0
12,496072,0.0,2.0,0.0
15,743959,48.0,24.0,0.0
19,1488291,7.0,0.0,0.0
21,247943,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1736086,65.0,11.0,8.0
1,12,496072,0.0,2.0,0.0
2,15,743959,48.0,24.0,0.0
3,19,1488291,7.0,0.0,0.0
4,21,247943,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,65.0,11.0,8.0
1,12,0.0,2.0,0.0
2,15,48.0,24.0,0.0
3,19,7.0,0.0,0.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,12,Released,0.0
2,15,Released,48.0
3,19,Released,7.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,8.0,0.0,0.0,0.0,0.0
Executing,11.0,2.0,24.0,0.0,1.0
Released,65.0,0.0,48.0,7.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,11.0,2.0,24.0,0.0,1.0
2,Released,65.0,0.0,48.0,7.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,11.0,2.0,24.0,0.0,1.0
2,Released,65.0,0.0,48.0,7.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()